In [1]:
%matplotlib inline
%load_ext autoreload

%autoreload 2
!pip install ../../tensortrade -U


Processing c:\users\codeninja\dev\tensortrade
  Created wheel for tensortrade: filename=tensortrade-0.0.1a18-cp37-none-any.whl size=55081 sha256=93e5e59791543c49ec9a0ac581ab7794eff450884e18c4387d0f70d72b2db426
  Stored in directory: C:\Users\CODENI~1\AppData\Local\Temp\pip-ephem-wheel-cache-g5nx0quw\wheels\f4\72\37\5f9b89cbcd3ef6d4590747c3505b6ed3c72fec82f02d22c51d
Successfully built tensortrade
  Found existing installation: tensortrade 0.0.1a18
    Uninstalling tensortrade-0.0.1a18:
      Successfully uninstalled tensortrade-0.0.1a18


In [2]:
import os
import sys
import warnings
import numpy

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 

sys.path.append(os.path.dirname(os.path.abspath('')))

**Read Historical Data**

Data generated using : https://github.com/codeninja/CCXT-Historical-Data

In [3]:
import pandas as pd

data_file ='./btc_usdt_1h.csv' 
df = pd.read_csv(data_file, index_col=[0])
df.head()

,symbol,open,high,low,close,volume
2019-09-30 07:00:00,BTC/USDT,7986.86,8014.21,7984.26,7998.49,269.915958
2019-09-30 08:00:00,BTC/USDT,7997.94,8152.80,7997.91,8114.76,4459.172673
2019-09-30 09:00:00,BTC/USDT,8114.99,8257.00,8090.42,8201.57,3209.899291
2019-09-30 10:00:00,BTC/USDT,8201.54,8300.00,8200.26,8283.85,5111.200055
2019-09-30 11:00:00,BTC/USDT,8284.01,8337.26,8232.05,8260.01,2893.741582


In [4]:
import ccxt

from tensortrade.strategies import TensorforceTradingStrategy

In [5]:
from tensortrade.rewards import SimpleProfitStrategy
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

# we only want to normalize the numeric fields, leaving 'symbol' untouched.
normalize = MinMaxNormalizer(columns=['open','close','high','low','volume'], inplace=True)

feature_pipeline = FeaturePipeline(steps=[normalize])

reward_strategy = SimpleProfitStrategy()
action_strategy = DiscreteActionStrategy(n_actions=20, instrument_symbol='BTC/USTD')

exchange = SimulatedExchange(data_frame=df,
                                should_pretransform_obs = True,
                                base_instrument='USDT', 
                                timeframe='1h',
                                commission_percent=0.5,
                                window_size=1,
                                max_allowed_slippage_percent=3.0,
                                min_order_amount=1E-4,
                                min_trade_amount=1E-4)

In [6]:
from tensortrade.environments import TradingEnvironment

network_spec = [
    dict(type='dense', size=64, activation="tanh"),
    dict(type='dense', size=32, activation="tanh")
]

agent_spec = {
    "type": "ppo",
    "learning_rate": 1e-4,
    "discount": 0.99,
    "likelihood_ratio_clipping": 0.2,
    "estimate_terminal": False,
    "max_episode_timesteps": 2000,
    "network": network_spec,
    "batch_size": 5,
    "update_frequency": "never"
}

environment = TradingEnvironment(exchange=exchange,
                                 action_strategy=action_strategy,
                                 reward_strategy=reward_strategy,
                                 feature_pipeline=feature_pipeline)

strategy = TensorforceTradingStrategy(environment=environment, agent_spec=agent_spec)

In [7]:
performance = strategy.run(episodes=10, evaluation=False)

performance[-5:]

Episodes:   0%|                                | 0/10 [00:00, reward=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%]

Empty DataFrame
Columns: [symbol, open, high, low, close, volume]
Index: []


TypeError: ufunc 'isinf' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
performance.balance.plot()

In [ ]:
strategy.save_agent(directory='agents')